## Setup

In [ ]:
! pip install import_ipynb
import import_ipynb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline    

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=2dba391cec0497df44db645731c20e7ee3b34b1fee5228c5c24d41f2b778eefe
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
# mount google drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/flu_predictions/curr'
import flu_preprocessing as prep
%cd '/content/drive/'

/content/drive/My Drive/Colab Notebooks/flu_predictions/curr
importing Jupyter notebook from flu_preprocessing.ipynb


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


/content/drive


In [ ]:
# get data from google drive and read in as csv
train_features = pd.read_csv('/content/drive/My Drive/sideprojects/flu_predictions/training_set_features.csv')
train_labels = pd.read_csv('/content/drive/My Drive/sideprojects/flu_predictions/training_set_labels.csv')
test_features = pd.read_csv('/content/drive/My Drive/sideprojects/flu_predictions/test_set_features.csv')

## Preprocessing

In [ ]:
# combine train_features and train_labels for preprocessing - drop extra respondent_id columns
train = pd.concat([train_features, train_labels.iloc[:,1:]], axis=1)

In [ ]:
# print('TRAINING DATA SUMMARY')
# train.info()
# print('TEST DATA SUMMARY')
# test_features.info()

### Basic cleaning

In [ ]:
# count null values 
# prep.nullFinder(train, 'TRAINING')
# prep.nullFinder(test_features, 'TEST')

# reorganize employment_industry, employment_occupation
# drop health insurance - half of its values are missing
prep.organize(train)
prep.organize(test_features)

In [ ]:
# # recheck the number of null values in employment_industry and employment_occupation - did manipulating these columns
# # based on employment_status help? 

# print('employment_industry nulls (TRAINING): {}'.format(
#     len(train['employment_industry']) - train['employment_industry'].count()))
# print('employment_occupation nulls (TRAINING): {}'.format(
#     len(train['employment_industry']) - train['employment_industry'].count()))
# print('employment_industry nulls (TEST): {}'.format(
#     len(test_features['employment_industry']) - test_features['employment_industry'].count()))
# print('employment_occupation nulls (TEST): {}'.format(
#     len(test_features['employment_industry']) - test_features['employment_industry'].count()))

In [ ]:
# drop duplicates, drop respondent_id because that won't help us classify this data
train.drop(['respondent_id'], axis=1, inplace=True)
test_features.drop(['respondent_id'], axis=1, inplace=True)

In [ ]:
# drop rows with null values
train.dropna(inplace=True)
train.reset_index(inplace=True, drop=True)

test_features.dropna(inplace=True)
test_features.reset_index(inplace=True, drop=True)

### Column engineering

| feature | new level | new level | new level |
| ------- | ------ | ------ | ------| 
| h1n1_concern | [(0,1) to become] 0 = less concern <br> | [(2,3) to become] 1 = more concern<br> | NONE | 
| household_adults (will be renamed: live_alone) | [(1,2,3) to become] 0 = does not live alone | [(0) to become] 1 = lives alone | NONE | 
| household_children (will be renamed: have_children) | [(0) to become] 0 = no children | [(1,2,3) to become] 1 = children | NONE | 
| employment_status | [(['Not in Labor Force', 'Unemployed']) to become] 0 = not working | [(['Employed']) to become] 1 = working | NONE |
| education | [(['< 12 Years', '12 Years']) to become] 0 = high school or less | [(['Some College']) to become] 1 = some college | [(['College Graduate']) to become] 2 = college grad |
| opinion_h1n1_vacc_effective | [(1,2) to become] 0 = not effective | [(3) to become] 1 = don't know | [(4,5) to become] 2 = effective | 
| opinion_seas_vacc_effective | [(1,2) to become] 0 = not effective | [(3) to become] 1 = don't know | [(4,5) to become] 2 = effective | 
|opinion_h1n1_sick_from_vacc | DROP 3 (DON'T KNOW) - too small, creates noise | [(1,2) to become] 0 = not worried | [(4,5) to become] 1 = worried |
| opinion_seas_sick_from_vacc  | DROP 3 (DON'T KNOW) - too small, creates noise | [(1,2) to become] 0 = not worried | [(4,5) to become] 1 = worried |
| opinion_h1n1_risk | DROP 3 (DON'T KNOW) - too small, creates noise | [(1,2) to become] 0 = low | [(4,5) to become] 1 = high |
| opinion_seas_risk | DROP 3 (DON'T KNOW) - too small, creates noise | [(1,2) to become] 0 = low | [(4,5) to become] 1 = high |

In [ ]:
# transforming columns based on the specifications above
train = prep.columnTransformer(train)
test_features = prep.columnTransformer(test_features)

### Encoding features
| Feature type | Feature description | Associated features | Action |
| ------------ | ------------------- | ------------------- | ------ |
| binary | 2 levels [0,1] - <br> described as no/yes, low/high, etc | - 'h1n1_concern' <br> - 'behavioral_avoidance' <br> - 'behavioral_wash_hands' <br> - 'behavioral_large_gatherings' <br>- 'behavioral_outside_home' <br> - 'behavioral_touch_face' <br> - 'doctor_hecc_h1n1' <br> - 'doctor_recc_seasonal' <br> - 'chronic_med_condition' <br> - 'health_worker' <br>- 'opinion_h1n1_risk' <br>- 'opinion_h1n1_sick_from_vacc' <br>- 'opinion_seas_risk' <br>- 'opinion_seas_sick_from_vacc' <br> - employment_status <br>- 'live_alone' <br> - 'have_children' <br>- 'education' <br> - 'behavioral_antiviral_meds' <br> - 'behavioral_face_mask' <br> - 'child_under_6_months' | No action - 2 levels each (0,1) |
| multiple | >2 levels [0,1,2,n...] | - 'opinion_h1n1_vacc_effective' <br>- 'opinion_seas_vacc_effective' <br> - 'h1n1_knowledge' | No action - 3 levels each | 
| text | levels are strings | - 'age_group' <br>- 'sex'<br>- 'income_poverty'<br>- 'rent_or_own'<br>- 'hhs_geo_region'<br>- 'census_msa'<br>- 'race'<br>- 'employment_industry'<br>- 'employment_occupation' <br>- marital_status | - census_msa and income_poverty to be manually ordinally encoded <br> - age_group to be ordinally encoded <br> - hhs_geo_region, race, employment_industry, employment_occupation <br>&nbsp; to be feature hash encoded OR binary encoded<br> - sex, marital_status, and rent_or_own to be one hot encoded (drop_first=True) | 


In [ ]:
# drop duplicates
train.drop_duplicates(ignore_index=True, inplace=True)
test_features.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
# split the features and labels again, before encoding
train = train.reset_index(drop=True)

train_features = train.iloc[:,:-2]
train_labels = train.iloc[:,-2:]

test_features = test_features

h1n1_labels = train_labels['h1n1_vaccine']
szn_labels = train_labels['seasonal_vaccine']

In [ ]:
train_features, test_features = prep.encoder(train_features, test_features, 'binary')

## Feature selection

#### Feature selections using VIF (ie evaluating multicollinearity)

In [ ]:
# turn all encoded features into pd.categorical type 
for t in list(train_features.columns):
  train_features[t] = pd.Categorical(train_features[t])
for t in list(test_features.columns):
  test_features[t] = pd.Categorical(test_features[t])

In [ ]:
# change dataframes to float type 
train_features = pd.DataFrame(np.array(train_features, dtype=float), columns=train_features.columns)
test_features = pd.DataFrame(np.array(test_features, dtype=float), columns=test_features.columns)

In [ ]:
train_vif = prep.checkVIF(train_features)
multico_features = list(train_vif[train_vif['VIF Factor']>=10.0]['features'])

/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
train_features = train_features.drop(multico_features, axis=1)
test_features = test_features.drop(multico_features, axis=1)

#### Feature selection using correlations


In [ ]:
# get correlation matrix 
trainfCorrs = prep.cramersV(train_features)

# create correlation heatmap
# prep.corrPlot(trainfCorrs, 'train data')

# correlation analysis
train_corrAnalysis, corr_drop = prep.corrAnalysis(trainfCorrs, 0.6)

In [ ]:
train_corrAnalysis

,compared_feature1,compared_feature2,feature_to_drop,cramersV_for_featuretoDrop
0,marital_status,lives_alone,marital_status,0.62


In [ ]:
# Drop the correlated columns we found above 
train_features = train_features.drop(corr_drop, axis=1)
test_features = test_features.drop(corr_drop, axis=1)

#### Remove unbalanced features

In [ ]:
# imbalanced = []
# for df in prep.colVals(train_features, plots=0):
#   if any(df.values<=10):
#     imbalanced.append(df.name)
#     # display(pd.DataFrame(df))

In [ ]:
# train_features = train_features.drop(imbalanced, axis=1)
# test_features = test_features.drop(imbalanced, axis=1)

#### Feature importances (using Random Forest)

##### train test split

In [ ]:
# make a copy of test_features because we will predict h1n1 and seasonal separately 
Xh_test = test_features.copy()
Xs_test = test_features.copy()

In [ ]:
# train validation split
Xh_train, Xh_val, yh_train, yh_val = train_test_split(train_features, h1n1_labels, test_size=0.2, random_state=0)
Xs_train, Xs_val, ys_train, ys_val = train_test_split(train_features, szn_labels, test_size=0.2, random_state=0)

##### implementing random forest

In [ ]:
# scale and balance 
Xh_train, yh_train = prep.scale(Xh_train, yh_train)
Xh_train, yh_train = prep.balance(Xh_train, yh_train)
Xh_val, yh_val = prep.scale(Xh_val, yh_val)
Xh_val, yh_val = prep.balance(Xh_val, yh_val)

Xs_train, ys_train = prep.scale(Xs_train, ys_train)
Xs_val, ys_val = prep.scale(Xs_val, ys_val)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
## randomized search setup 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# create pipeline
rf_pipe = Pipeline([('classifier', RandomForestClassifier(random_state=0))])

# create search space
rf_params1 = [{'classifier__max_depth': [10,20],
              'classifier__min_samples_split': [6,10], 
              'classifier__max_leaf_nodes': [75,150],  
              'classifier__n_estimators': [250,750],  
              'classifier__max_features': ['sqrt', 0.5, 1.0]}]
rf_params2 = [{'classifier__max_depth': [5,10,15,20,25],
              'classifier__min_samples_split': [4, 6, 8, 10, 12], 
              'classifier__max_leaf_nodes': [50, 100, 250, 500, 750],  
              'classifier__n_estimators': [100, 250, 500, 750],  
              'classifier__max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
              'classifier__bootstrap': [False, True],}]

# create the searcher
hrf_search = RandomizedSearchCV(
    estimator = rf_pipe,
    param_distributions = rf_params1,
    n_iter = 10,
    scoring=['f1', 'roc_auc'],
    refit='roc_auc',
    n_jobs=-1,
    cv=4,
    verbose=2)
srf_search = RandomizedSearchCV(
    estimator = rf_pipe,
    param_distributions = rf_params1,
    scoring=['f1', 'roc_auc'],
    refit='roc_auc',
    n_jobs=-1,
    cv=4,
    verbose=2)

In [ ]:
# fit the searcher object 
# hrf_search.fit(Xh_train, yh_train)
# srf_search.fit(Xs_train, ys_train)


# lr_pipe = Pipeline([('classifier', LogisticRegression())])
# {'classifier': [LogisticRegression()],
#                  'classifier__penalty': ['l1', 'l2'],
#                  'classifier__C': np.logspace(0, 4, 10)}


In [ ]:
# import torch
# model_save_name = 'srf_corrVIF_noimb.pt'
# path = F"/content/drive/My Drive/{model_save_name}"
# torch.save(srf_search, path)

import torch
hrf_search = torch.load(F"/content/drive/My Drive/hrf_corrVIF_noimb.pt")
srf_search = torch.load(F"/content/drive/My Drive/srf_corrVIF_noimb.pt")

In [ ]:
hrf_keys = [h[12:] for h in hrf_search.best_params_.keys()]
hrf_bestParams = {hrf_keys[i]: list(hrf_search.best_params_.values())[i] for i in range(len(hrf_keys))} 
hrf = RandomForestClassifier(random_state=0, **hrf_bestParams)
hrf.fit(Xh_train, yh_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='sqrt',
                       max_leaf_nodes=150, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, n_estimators=750,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
srf_keys = [s[12:] for s in srf_search.best_params_.keys()]
srf_bestParams = {srf_keys[i]: list(srf_search.best_params_.values())[i] for i in range(len(srf_keys))} 
srf = RandomForestClassifier(random_state=0, **srf_bestParams)
srf.fit(Xs_train, ys_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=0.5,
                       max_leaf_nodes=150, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, n_estimators=750,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
# feature importances
hrf_imp = pd.DataFrame(hrf.feature_importances_, 
                       index = Xh_train.columns, 
                       columns=['importance']).sort_values('importance', ascending=False)
srf_imp = pd.DataFrame(srf.feature_importances_, 
                       index = Xs_train.columns, 
                       columns=['importance']).sort_values('importance', ascending=False)

In [ ]:
hrf_feats = list(hrf_imp[hrf_imp['importance']>=0.01].index)
srf_feats = list(srf_imp[srf_imp['importance']>=0.005].index)

In [ ]:
Xh_train = Xh_train[hrf_feats]
Xh_val = Xh_val[hrf_feats]

Xs_train = Xs_train[srf_feats]
Xs_val = Xs_val[srf_feats]

## Modeling

In [ ]:
from sklearn.metrics import roc_auc_score, fbeta_score

##### rf

In [ ]:
hrf_keys = [h[12:] for h in hrf_search.best_params_.keys()]
hrf_bestParams = {hrf_keys[i]: list(hrf_search.best_params_.values())[i] for i in range(len(hrf_keys))} 
hrf = RandomForestClassifier(random_state=0, **hrf_bestParams)
hrf.fit(Xh_train, yh_train)

hpred = hrf.predict(Xh_val)
print(fbeta_score(yh_val, hpred, 1))
print(roc_auc_score(yh_val, hpred))

## 0.1
# 0.8577068006459716
# 0.8611208406304729

0.8577068006459716
0.8611208406304729


In [ ]:
srf_keys = [s[12:] for s in srf_search.best_params_.keys()]
srf_bestParams = {srf_keys[i]: list(srf_search.best_params_.values())[i] for i in range(len(srf_keys))} 
srf = RandomForestClassifier(random_state=0, **srf_bestParams)
srf.fit(Xs_train, ys_train)

spred = srf.predict(Xs_val)
print(fbeta_score(ys_val, spred, 1))
print(roc_auc_score(ys_val, spred))

## 0.05 
# 0.7630090497737556
# 0.7754732131753835

0.7630090497737556
0.7754732131753835


##### lightgbm

In [ ]:
! pip install bayesian-optimization

import lightgbm as lgb
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=b670f174dfc1a6b1025d6700591982d5f916279ad8aab681ab3c1292577ce6a8
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
# # conversion to lightgbm datasets
# Xht = lgb.Dataset(Xh_train, label = yh_train)
# Xst = lgb.Dataset(Xs_train, label = ys_train)

In [ ]:
model1 = lgb.LGBMClassifier(objective='binary', random_state=0)
model1.fit(Xh_train, yh_train)
predictions1 = model1.predict(Xh_val)
auc1 = roc_auc_score(yh_val, predictions1)
fbeta1 = fbeta_score(yh_val, predictions1, 1)
print('The baseline ROC AUC score on Xh_val is {:.3f}.'.format(auc1))
print('The baseline F1 score on Xh_val is {:.3f}.'.format(fbeta1))
print('\n')

model2 = lgb.LGBMClassifier(objective='binary', random_state=0)
model2.fit(Xs_train, ys_train)
predictions2 = model2.predict(Xs_val)
auc2 = roc_auc_score(ys_val, predictions2)
fbeta2 = fbeta_score(ys_val, predictions2, 1)
print('The baseline ROC AUC score on Xs_val is {:.3f}.'.format(auc2))
print('The baseline F1 score on Xs_val is {:.3f}.'.format(fbeta2))

The baseline ROC AUC score on Xh_val is 0.885.
The baseline F1 score on Xh_val is 0.880.


The baseline ROC AUC score on Xs_val is 0.775.
The baseline F1 score on Xs_val is 0.762.


In [ ]:
def bayes_parameter_opt_lgb(X, y, init_round=15, learning_rate=0.05, opt_round=25, n_folds=5, random_seed=0, 
                            n_estimators=250, output_process=False, is_unbalance=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, categorical_feature = 'auto', free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, 
                 learning_rate, num_iterations, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary',
                  'early_stopping_round':25, 
                  'metric':'auc',
                  'is_unbalance': is_unbalance}
        params['num_iterations'] = int(round(n_estimators))
        params["learning_rate"] = max(learning_rate, 0.01)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =0, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'num_iterations': (100,1000),
                                            'learning_rate': (0.01, 1),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.4, 1),
                                            'max_depth': (5, 12),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    return lgbBO.max

h_best = bayes_parameter_opt_lgb(Xh_train, yh_train, is_unbalance=False)
s_best = bayes_parameter_opt_lgb(Xs_train, ys_train, is_unbalance=False)

h_best2 = bayes_parameter_opt_lgb(Xh_train, yh_train, is_unbalance=True)
s_best2 = bayes_parameter_opt_lgb(Xs_train, ys_train, is_unbalance=True)  

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_sp... | num_it... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  1        |  0.9426   |  0.7293   |  0.6722   |  3.014    |  1.635    |  0.4294   |  9.521    |  24.69    |  0.08929  |  967.3    |  32.05    |


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  2        |  0.9451   |  0.875    |  0.5231   |  2.84     |  2.777    |  0.08033  |  5.61     |  5.91     |  0.08343  |  800.3    |  42.27    |


...

|  38       |  0.8409   |  0.9656   |  0.2146   |  4.199    |  2.081    |  0.2314   |  11.89    |  12.85    |  0.01187  |  198.8    |  25.63    |


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  39       |  0.8416   |  0.5585   |  0.5624   |  4.993    |  1.963    |  0.08126  |  10.88    |  44.9     |  0.07138  |  594.1    |  43.56    |


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  40       |  0.8395   |  0.9432   |  0.3968   |  3.97     |  1.24     |  0.6823   |  6.649    |  49.93    |  0.02249  |  982.6    |  44.23    |


In [ ]:
# h_best, etc --> is_unbalance = False
# h_best2, etc --> is_unbalance = True 

s_best

{'params': {'bagging_fraction': 0.8256852405426329,
  'feature_fraction': 0.4659199701154856,
  'lambda_l1': 1.3324476049641216,
  'lambda_l2': 2.828773432360044,
  'max_depth': 5.545409025492388,
  'min_child_weight': 5.2599158373575845,
  'min_split_gain': 0.01445070766280266,
  'num_leaves': 24.451808603241865},
 'target': 0.8424194790311758}